In [ ]:
import re
import os
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
from utils.common_utils import print_help
from ioc_processing.ioc_functions import (
    validate_iocs,
    parse_bulk_iocs,
    process_individual_ioc_file,
    analysis,
    calculate_total_malicious_score
)
from file_operations.file_utils import read_file, clean_input, is_ip, is_url, is_domain, is_hash


# Function to list files in the input_files directory
def list_input_files():
    input_dir = 'input_files'
    return [f for f in os.listdir(input_dir) if os.path.isfile(os.path.join(input_dir, f))]

# Inject custom CSS to change the background color and text color
custom_css = """
<style>
    .custom-box {
        background-color: #333333;
        padding: 20px;
        border-radius: 10px;
    }
    .widget-label-basic { 
        color: var(--custom-text-color, white) !important; 
    }
    .widget-radio-box, .widget-button, .widget-checkbox label {
        background-color: #444444 !important; 
        color: var(--custom-text-color, white) !important; 
    }
    .widget-textarea {
        color: var(--custom-text-color, white) !important;
    }
    .widget-button {
        border: 1px solid #555555 !important;
    }
</style>
"""
display(HTML(custom_css))

# Check for light or dark mode and update text color accordingly
mode_check_script = """
<script>
    function updateTextColor() {
        const colorMode = window.matchMedia('(prefers-color-scheme: dark)').matches ? 'dark' : 'light';
        const root = document.documentElement;
        root.style.setProperty('--custom-text-color', colorMode === 'dark' ? 'white' : 'black');
    }

    updateTextColor();
    window.matchMedia('(prefers-color-scheme: dark)').addEventListener('change', updateTextColor);
</script>
"""
display(HTML(mode_check_script))

# Add the regex patterns for each type of IOC
ip_regex = re.compile(r'^(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.([0-9]{1,3})\.([0-9]{1,3})\.([0-9]{1,3})$')
url_regex = re.compile(r'^(https?:\/\/)?([\w\d-]+\.)+[\w-]+(\/[\w-]*)*\/?$')
hash_regex = re.compile(r'^[a-f0-9]{32}$|^[a-f0-9]{40}$|^[a-f0-9]{64}$')  # MD5, SHA1, SHA256

# Function to validate the IOC based on the selected type
def validate_ioc_input(choice, ioc_input):
    if choice == '1':  # IP
        return all(ip_regex.match(ioc) for ioc in ioc_input.splitlines())
    elif choice == '2':  # URL/Domain
        return all(url_regex.match(ioc) for ioc in ioc_input.splitlines())
    elif choice == '3':  # Hash
        return all(hash_regex.match(ioc) for ioc in ioc_input.splitlines())
    return True  # For bulk, skip validation


# Function to handle validation and display results
def start_validation(choice, output_to_file, raw_input=None, file_input=None, file_name_input=None):
    output_file_path = None
    progress_bar = widgets.IntProgress(value=0, min=0, max=1, description='Progress:', bar_style='info', layout=widgets.Layout(width='100%', height='30px', margin='0 0 0 -20px'))

    with status_output:
        clear_output()
        display(HTML('<b>Processing, please wait...</b>'))
        display(progress_bar)

    if output_to_file:
        output_file_path = file_input or file_name_input

    ioc_type = {'1': 'ips', '2': 'urls', '3': 'hashes'}[choice]
    iocs = None

    if file_input:
        with status_output:
            clear_output()
            display(HTML('<b>Processing individual IOC file...</b>'))
            display(progress_bar)
        iocs = process_individual_ioc_file(file_input, ioc_type)
    elif file_name_input:
        with status_output:
            clear_output()
            display(HTML('<b>Processing individual IOC file...</b>'))
            display(progress_bar)
        iocs = process_individual_ioc_file(file_name_input, ioc_type)
    elif raw_input:
        cleaned_content = clean_input(raw_input)
        iocs = {ioc_type: cleaned_content.splitlines()}

    print(f"DEBUG: Parsed IOCs = {iocs}")

    # Validation to ensure correct IOC type is entered
    if choice == '1':  # IPs selected
        if any(not is_ip(ioc) for ioc in iocs['ips']):
            with status_output:
                clear_output()
                display(HTML("<b>Error: You selected 'IP' but entered a URL or hash. Please reset and enter a valid IP address.</b>"))
            return
    elif choice == '2':  # URLs/Domains selected
        if any(not is_url(ioc) for ioc in iocs['urls']):
            with status_output:
                clear_output()
                display(HTML("<b>Error: You selected 'URL/Domain' but entered an IP or hash. Please reset and enter a valid URL or domain.</b>"))
            return
    elif choice == '3':  # Hashes selected
        if any(not is_hash(ioc) for ioc in iocs['hashes']):
            with status_output:
                clear_output()
                display(HTML("<b>Error: You selected 'Hash' but entered an IP or domain. Please reset and enter a valid hash.</b>"))
            return

    if not iocs or not iocs.get(ioc_type, []):
        with status_output:
            clear_output()
            display(HTML('<b>Error: No valid IOCs found for the selected IOC type.</b>'))
        return

    progress_bar.max = len(iocs[ioc_type]) * 6

    with status_output:
        clear_output()
        display(progress_bar, HTML('<b>Performing analysis...</b>'))

    try:
        # Corrected call: Passing `ioc_type` as the selected_category and `iocs` as parsed IOCs
        print(f"DEBUG: Starting analysis with ioc_type = {ioc_type} and IOCs = {iocs}")
        aggregated_report = analysis(ioc_type, iocs, output_file_path, progress_bar, status_output)

        if output_to_file:
            with open(output_file_path, "a") as outfile:
                outfile.write(aggregated_report)
                outfile.write("\n")

        with output:
            clear_output()
            print(aggregated_report)

        with status_output:
            clear_output()
            display(progress_bar)
            display(HTML('<b>Processing complete.</b>'))

    except TypeError as e:
        with status_output:
            clear_output()
            display(HTML(f"<b>Error occurred during analysis: {str(e)}</b>"))
        print(f"DEBUG: Error Traceback: {str(e)}")

# Function to reset the inputs and output
def reset_inputs(b):
    ioc_type_radio.value = '1'
    output_to_file_checkbox.value = False
    file_input.value = ()  # Reset file input
    file_name_input.value = None  # Reset dropdown selection
    raw_input.value = ''
    with output:
        clear_output()
    with status_output:
        clear_output()
    processing_label.value = ""  # Clear the processing message
    print("Inputs have been reset.")  # Debugging message

# Create widgets with custom styles
style = {'description_width': 'initial'}
layout = widgets.Layout(width='auto')

ioc_type_radio = widgets.RadioButtons(
    options=[('IPs', '1'), ('URLs/Domains', '2'), ('Hashes', '3')],
    description='IOC Type:',
    disabled=False,
    style=style,
    layout=widgets.Layout(width='250px', padding='10px', background_color='#333333', border_radius='10px')
)

output_to_file_checkbox = widgets.Checkbox(
    value=False,
    description='Save output to file',
    disabled=False,
    style=style,
    layout=widgets.Layout(width='250px', padding='10px', background_color='#333333', border_radius='10px')
)

file_input = widgets.FileUpload(
    accept='.txt',  # Accept only .txt files
    multiple=False,  # Accept single file
    style=style,
    layout=widgets.Layout(width='250px')
)

file_name_input = widgets.Dropdown(
    options=list_input_files(),
    value=None,
    placeholder='Select file from input_files directory',
    description='File Name:',
    disabled=False,
    style=style,
    layout=widgets.Layout(width='250px', background_color='transparent')
)

raw_input = widgets.Textarea(
    value='',
    placeholder='Type raw IOCs here, separated by new lines',
    description='Raw Input:',
    disabled=False,
    style=style,
    layout=widgets.Layout(width='250px', background_color='transparent')
)

submit_button = widgets.Button(
    description='Start Validation',
    disabled=False,
    button_style='info',  # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to start validation',
    icon='check',  # (FontAwesome names without the `fa-` prefix)
    style=style,
    layout=widgets.Layout(width='250px')
)

reset_button = widgets.Button(
    description='Reset',
    disabled=False,
    button_style='warning',
    tooltip='Click to reset inputs',
    icon='refresh',
    style=style,
    layout=widgets.Layout(width='250px')
)

processing_label = widgets.Label(value="", layout=widgets.Layout(width='250px', margin='-5px 0 0 0'))  # Label to indicate processing status

output = widgets.Output(layout=widgets.Layout(width='100%', height='900px', overflow='auto'))  # Increased height to 900px
status_output = widgets.Output(layout=widgets.Layout(width='100%', height='100px'))  # Adjusted height to fit status and progress bar

# Define the interaction logic
def on_submit_button_clicked(b):
    choice = ioc_type_radio.value
    output_to_file = output_to_file_checkbox.value
    raw_text = raw_input.value.strip()  # Get raw text input and trim whitespace
    file_name = None
    file_name_text = file_name_input.value

    if file_input.value:
        file_name = list(file_input.value.values())[0]['metadata']['name']
        with open(file_name, 'wb') as f:
            f.write(file_input.value[list(file_input.keys())[0]]['content'])

    # Validate the input based on the selected radio button (IP, URL/Domain, Hash)
    if choice in ['1', '2', '3']:  # Only validate for individual types
        if not raw_text and not file_name and not file_name_text:
            with status_output:
                clear_output()
                print("Error: Please provide an IOC input or select a file.")
            return
        
        # Validate the raw text input or file content
        if raw_text and not validate_ioc_input(choice, raw_text):
            with status_output:
                clear_output()
                print("Error: Invalid input for IOC type.\nPlease reset and enter valid IOC for the type selected.")
            return

    start_validation(choice, output_to_file, raw_text, file_name, file_name_text)

# Bind the button click events to the functions
submit_button.on_click(on_submit_button_clicked)
reset_button.on_click(reset_inputs)

# Arrange the UI components in a box with a grey background
input_widgets = widgets.VBox([
    widgets.VBox([ioc_type_radio], layout=widgets.Layout(padding='10px', background_color='#333333', border_radius='10px')),
    widgets.VBox([output_to_file_checkbox], layout=widgets.Layout(padding='10px', background_color='#333333', border_radius='10px')),
    file_input,
    file_name_input,
    raw_input,
    submit_button,
    reset_button,
    processing_label,
    status_output,
], layout=widgets.Layout(border='1px solid #ccc', padding='10px', background_color='#333333', border_radius='10px'))

app_layout = widgets.AppLayout(
    left_sidebar=input_widgets,
    center=None,
    right_sidebar=output,
    pane_widths=['300px', 0, 1],  # Reduced width of the left sidebar
    pane_heights=['80px', '900px', '60px']  # Increased height to 900px
)

display(app_layout)